In [1]:
# importando as bibliotecas
import gurobipy as gp
from gurobipy import GRB

In [2]:
# conjuntos: recursos e trabalhos
I = ['Carlos', 'Jonas', 'Monica']
J = ['Cientista', 'Desenvolvedor', 'Arquiteto']

In [3]:
# matriz percentual
key, perc = gp.multidict({
    ('Carlos', 'Cientista'): 53,
    ('Carlos', 'Desenvolvedor'): 27,
    ('Carlos', 'Arquiteto'): 13,
    ('Jonas', 'Cientista'): 80,
    ('Jonas', 'Desenvolvedor'): 47,
    ('Jonas', 'Arquiteto'): 67,
    ('Monica', 'Cientista'): 53,
    ('Monica', 'Desenvolvedor'): 73,
    ('Monica', 'Arquiteto'): 47
})

In [24]:
print(key)

('Ana', 'Arquiteto')


In [22]:
perc

{('Carlos', 'Cientista'): 53,
 ('Carlos', 'Desenvolvedor'): 27,
 ('Carlos', 'Arquiteto'): 13,
 ('Jonas', 'Cientista'): 80,
 ('Jonas', 'Desenvolvedor'): 47,
 ('Jonas', 'Arquiteto'): 67,
 ('Monica', 'Cientista'): 53,
 ('Monica', 'Desenvolvedor'): 73,
 ('Monica', 'Arquiteto'): 47}

In [4]:
# definindo o modelo
m = gp.Model("atribuicao")

Academic license - for non-commercial use only - expires 2022-09-29
Using license file /opt/gurobi912/gurobi.lic


In [5]:
# adicionando variaveis
x = m.addVars(key, name='x')

In [6]:
# adicionando restricoes de trabalho
trabalho = m.addConstrs((x.sum('*',j) == 1 for j in J),'trabalho')

In [7]:
# adicionando restricoes de recursos(pessoas)
pessoa = m.addConstrs((x.sum(i,'*') <= 1 for i in I),'pessoa')

In [8]:
# definindo a funcao objetivo
m.setObjective(x.prod(perc),GRB.MAXIMIZE)

In [10]:
# exportando .lp/.mps
m.write('atribuicao.mps')

In [11]:
# resolvendo o problema
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xb343b6eb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+32   1.800000e+31   4.600000e+02      0s
       5    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds
Optimal objective  1.930000000e+02


In [13]:
#imprimindo solucao
for var in m.getVars():
    if abs(var.x) > 1e-6:
        print(var.varName, '=', var.x)

x[Carlos,Cientista] = 1.0
x[Carlos,Desenvolvedor] = 0.0
x[Carlos,Arquiteto] = 0.0
x[Jonas,Cientista] = 0.0
x[Jonas,Desenvolvedor] = 0.0
x[Jonas,Arquiteto] = 1.0
x[Monica,Cientista] = 0.0
x[Monica,Desenvolvedor] = 1.0
x[Monica,Arquiteto] = 0.0


In [14]:
# imprimindo valor objetivo
print('Percentual =', m.objVal)

Percentual = 193.0


In [15]:
# alterando o cenario. Ana passa a fazer parte da selecao
nperc = {('Ana','Cientista'):100, ('Ana','Desenvolvedor'):100, ('Ana','Arquiteto'):100}

In [16]:
# alterando o cenario. Ana passa a fazer parte da selecao
for key, val in nperc.items():
    i, j = key
    x[key] = m.addVar(obj=val,
                    name='x[{0},{1}]'.format(i,j),
                    column=gp.Column([1],[m.getConstrByName('trabalho[{0}]'.format(j))])
                   )

In [17]:
# alterando o cenario. Ana passa a fazer parte da selecao
m.addConstr(x.sum('Ana','*') <= 1, name='pessoa[Ana]')

<gurobi.Constr *Awaiting Model Update*>

In [18]:
# exportando .lp/.mps
m.write('atribuicao_novo.mps')

In [19]:
# resolvendo o problema
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 12 columns and 24 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4100000e+32   9.000000e+30   1.410000e+02      0s
       2    2.5300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  2.530000000e+02


In [20]:
# imprimindo solucao
for var in m.getVars():
    if abs(var.x) > 1e-6:
        print(var.varName, '=', var.x)

x[Jonas,Cientista] = 1.0
x[Monica,Desenvolvedor] = 1.0
x[Ana,Arquiteto] = 1.0


In [21]:
# imprimindo o valor otimo
print("Novo percentual = ", m.objVal)

Novo percentual =  253.0
